In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.4 MB/s eta 0:00:00:00:0100:01


In [ ]:
from google.colab import auth
auth.authenticate_user()

import os
token = input("Hugging Face token:")
os.environ["HUGGINGFACE_HUB_TOKEN"] = token

In [3]:
import pandas as pd
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import time

In [4]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,                  
    torch_dtype=torch.float16,
    token=token
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

2025-06-03 01:24:34.045198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748913874.228704      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748913874.281082      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [6]:
# Data Loader
df = pd.read_csv("/kaggle/input/raw-address/address_full_0712.csv")
addresses = df["Address"].dropna().tolist()
output_file = "1883-4500.json"

Ở đây chỉ prompt cho LLM sửa một số lỗi như viết tắt (như q. -> quận, p.-> phường,...), nhận dạng sơ qua các đơn vị hành chính, đối với một số câu không có tên tỉnh/tên thành phố, sẽ không được generate thêm vì tỉnh/thành phố có thể duy nhất, nhưng tên quận/huyện/thị trấn là không duy nhất => sai địa chỉ trầm trọng hơn => có thể ảnh hưởng đến chi phí vận hành cũng như có nhiều sai lệch trong các quá trình xử lý/sử dụng data

In [ ]:
# Set processing range - CHANGE THESE VALUES
start_index = 4500
end_index = 5529

# Validate range
start_index = max(0, start_index)
end_index = min(end_index, len(addresses))

print(f"Total addresses: {len(addresses)}")
print(f"Processing addresses from index {start_index} to {end_index-1} ({end_index - start_index} addresses)")

# Process specified range
for i in range(start_index, end_index):
    addr = addresses[i]

    messages = [
        {
            "role": "system",
            "content": (
                "Bạn là chuyên gia xử lý và chuẩn hóa địa chỉ thô tại Việt Nam.\n"
                "Nhiệm vụ của bạn là chuẩn hóa địa chỉ thành định dạng hành chính đúng quy tắc.\n"
                "- Viết hoa đúng tên riêng và danh xưng hành chính (Phường, Xã, Quận, Huyện, Tỉnh, TP, ...).\n"
                "- Chỉ sử dụng thông tin có trong đầu vào. Không được tự thêm địa danh hoặc cấp hành chính nếu không có rõ trong chuỗi gốc.\n"
                "- Nếu không có đủ 5 cấp, vẫn giữ nguyên các phần có trong địa chỉ đầu vào.\n"
                "- Kết quả trả về là một chuỗi string duy nhất là địa chỉ đã chuẩn hóa. Không được thêm giải thích, văn bản thừa, hay placeholder."
            )
        },
        {
            "role": "user",
            "content": "mizuki , khu đô thị liền kề"
        },
        {
            "role": "assistant",
            "content": "Mizuki, Khu Đô Thị Liền Kề"
        },
        {
            "role": "user",
            "content": "123/5 đường lê văn sỹ, phường 13, quận 3, tp hà nội"
        },
        {
            "role": "assistant",
            "content": "Số 123/5, Đường Lê Văn Sỹ, Phường 13, Quận 3, Thành phố Hà Nội"
        },
        {
            "role": "user",
            "content": "ấp bình thuận, xã tân phú, huyện củ chi, tp hcm"
        },
        {
            "role": "assistant",
            "content": "Ấp Bình Thuận, Xã Tân Phú, Huyện Củ Chi, Thành phố Hồ Chí Minh"
        },
        {
            "role": "user",
            "content": "khu phố 6, p. linh trung, q. thủ đức"
        },
        {
            "role": "assistant",
            "content": "Khu Phố 6, Phường Linh Trung, Quận Thủ Đức"
        },
        {
            "role": "user",
            "content": "him lam phú an"
        },
        {
            "role": "assistant",
            "content": "Him Lam Phú An"
        },
        {
            "role": "user",
            "content": f"{addr}"
        }
    ]

    # Tokenize and generate
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    result = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()

    output_json = {
            "Id": i,
            "Sentence": result
        }

    # Save
    with open(output_file, "a", encoding="utf-8") as f:
        f.write(json.dumps(output_json, ensure_ascii=False) + "\n")

    print(f"Processed {i+1}/{end_index}: {result}")

    time.sleep(0.5)

Total addresses: 5529
Processing addresses from index 4500 to 5528 (1029 addresses)
Processed 4501/5529: Thủ Đức - Legacy Central
Processed 4502/5529: Mặt Tiền Đường Thuận Giao 25, Phường Thuận Giao, Thành phố Thuận An
Processed 4503/5529: Giai Việt
Processed 4504/5529: Số 856, Tạ Quang Bửu, Phường 5, Quận 8
Processed 4505/5529: Hẻm 2 Sẹc Hòa Bình, Phường Hiệp Tân, Quận Tân Phú
Processed 4506/5529: Đường Nguyễn Văn Quá, Quận 12
Processed 4507/5529: Mặt Tiền Đường Thạnh Lộc 15, Phường An Phú Đông
Processed 4508/5529: Phố Phùng Chí Kiên, Quận Cầu Giấy, Hà Nội
Processed 4509/5529: Làng Đại Học Thủ Đức, Hồ Chí Minh
Processed 4510/5529: Hòa Sơn - Lương Sơn - Hòa Bình
Processed 4511/5529: Tháp T3, Estella Heights
Processed 4512/5529: Đường Nguyễn Văn Quá, Quận 12
Processed 4513/5529: Chung Cư Phú Mỹ View, Cầu Phú Mỹ, Landmark 81, Bitexco
Processed 4514/5529: Đất Phước Bình, Long Thành, Đồng Nai
Processed 4515/5529: Oriental Plaza 685, Âu Cơ, Tân Phú
Processed 4516/5529: Cổng Chính Bệnh Viện 